<a href="https://colab.research.google.com/github/Hernanros/NLP-Ydata/blob/master/HW2/NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Text generation by Markov chain

In [0]:
import csv
import pandas as pd

Markov chain is a probabalistic model in which the probability of each event depends only on the state attained in the previous event. [markovify](https://github.com/jsvine/markovify) is a library for text generation by Markov chain.

Use "pip install markovify" to install markovify

In [2]:
!pip install markovify
import markovify

     |████████████████████████████████| 245kB 5.1MB/s 
  Created wheel for markovify: filename=markovify-0.8.0-cp36-none-any.whl size=10694 sha256=424e47b294acd429bb9c29f1c04645c49aa3b38ee9fba300e146420ef8bb2432
  Stored in directory: /root/.cache/pip/wheels/5d/a8/92/35e2df870ff15a65657679dca105d190ec3c854a9f75435e40
Successfully built markovify


In [0]:
def parseJson(fname):
    for line in open(fname, 'r+'):
        yield eval(line)

data = pd.DataFrame(list(parseJson('Sarcasm_Headlines_Dataset_v2.json')))



data.is_sarcastic = data.is_sarcastic.astype('bool')


In [6]:
data.head()

,is_sarcastic,headline,article_link
0,True,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,False,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,False,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,True,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,True,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [7]:
len([headline for i,headline in enumerate(data.headline) if data.is_sarcastic[i]==True])

13634

Download [Dataset.csv](https://drive.google.com/file/d/1raxIkJZ4lMTvgTB8eYjxu4Ux8aNL-x0s/view?usp=sharing) composed of sarcastic and serious headlines for the news. The csv-file consists of two columns. "headline" column contains texts of headlines. "is_sarcastic" column contain 0 if the hiadline is serious and 1 otherwise.

Read dataset 

In [8]:
texts_serious = [headline for i,headline in enumerate(data.headline) if data.is_sarcastic[i]==False] # list of serious hiadline texts
texts_sarcastic = [headline for i,headline in enumerate(data.headline) if data.is_sarcastic[i]==True] # list of sarcastic hiadline texts
# with open('Dataset.csv', encoding='utf-8') as f:
#     # Rread csv-file by DictReader from csv library
#     reader = csv.DictReader(f) 
#     for line in reader:
#         # read texts of headline
#         headline = line['headline'].strip()
#         # read sarcasticity of headline
#         is_sarcastic = int(line['is_sarcastic'].strip())
#         if is_sarcastic:
#             texts_sarcastic.append(headline)
#         else:
#             texts_serious.append(headline)
print('Found {} sarcastic texts in Dataset'.format(data.groupby('is_sarcastic').size()[1]))
print('Found {} serious texts in Dataset'.format(data.groupby('is_sarcastic').size()[0]))


Found 13634 sarcastic texts in Dataset
Found 14985 serious texts in Dataset


Merge headlines into one text

In [0]:
text_0 = '\n '.join(texts_serious)
text_1 = '\n '.join(texts_sarcastic)

In [7]:
markovify.Text(texts_sarcastic)

Create Markov chain model for serious headlines generation

In [0]:
serious_model = markovify.Text(texts_serious, state_size=2)
sarcastic_model = markovify.Text(texts_sarcastic)


Generate 20 serious headlines

In [15]:
for i in range(20):
    print(serious_model.make_sentence())

how to re-ignite the spark in your 20s: how to rekindle a lost passion in your young children
how 3 badass women stopped an alleged chemical weapon attack in idlib
why changing the way we talk about gray
democrats score special election results roll in
google translate update may save you from finding calm
john legend speaks out against gun violence in myanmar
slaughtering wolves in canada: a new chapter in u.s.-cuba relations
congresswoman says former mexican president
as i watch my daughter with the world.
south african prison rape this holiday season
they wore it best: miami's most glam moments of lgbtq icons in stunning catch and release
so you're hiring a web design company
meet the girl scout movement to raise a glass to pink for gleefully slamming the troll who hacked dnc reportedly target u.s. nuclear plants
palestinian shot dead a month after newtown. this is what earth would be
amanda slavin: not just for college grads in 2015
dog missing for almost a decade reunites with col

Create model and generate 20 sarcastic headlines

In [16]:
for i in range(20):
    print(sarcastic_model.make_sentence())

president's american manufacturing council down to single tofurkey
holocaust film appeals to hispanic voters for return of bachmann's full attention
man running toward departing train must have killed great white shark ride
texas to execute death row inmates could decimate texas restaurant industry not that original
man runs out of this
nation surprised to see people as human shields
ivana trump calls ex-husband to ask them how they could be last date in ages lets dangling modifier slide
bono outbids everyone at work waiting to see the view from apartment's roof
area woman always has backup problem just in case rest of year announced
new study finds 85% of americans just want to give baby up for adoption
man can get by in his state still effectively subhuman
weary, cynical woman knows to look directly at sun
well known gresham, or musicians form gresham, or musicians form gresham, or supergroup
uneasy détente forms between man sitting on shower wall can't believe he's filming another b

## Text generation by Variation Autoencoder

This part of tutorial based on [Text generation with a Variational Autoencoder](https://nicgian.github.io/text-generation-vae/) article. For sentence generation we use Variational Autoencoder (VAE) neural network model that is an extension seq2seq model. Originally VAE was described in [Auto-Encoding Variational Bayes](https://arxiv.org/pdf/1312.6114.pdf) paper. The idea behind Variational Autoencoder is that we impose predefined disribution (e.g., normal distribution) on the latent state formed by encoder. On the one hand this restriction alow us to sample random vectors from normal distribution and generate arbitrary sentences. On the othe hand this restriction form very dense well differentiated space without holes

In [10]:
import tensorflow as tf
import tensorflow_addons as tfa


from keras.layers import Bidirectional, Dense, Embedding, \
Input, Lambda, LSTM, RepeatVector, TimeDistributed, Layer, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers.advanced_activations import ELU
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model

from scipy import spatial
import pandas as pd
import numpy as np
import codecs
import random
import csv
import os

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [11]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Dowload [GloVe](http://nlp.stanford.edu/data/glove.6B.zip) pretrained word embedding vectors

In [12]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-05-24 12:46:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-24 12:46:52--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-24 12:46:52--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [13]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
MAX_SEQUENCE_LENGTH = 15 # Max text length in tokens
MAX_NB_WORDS = 20000 # Max words in dictionary
EMBEDDING_DIM = 50 # Dimensionality of GloVe vectors 

GLOVE_EMBEDDING = 'glove.6B.50d.txt'

train_data, val_data = train_test_split(data, test_size = .25, random_state= 42)
train_data.to_csv('train_data.csv')
val_data.to_csv('val_data.csv')


Create sentence tokenizer and two dictionaries: word_to_id and id_to_word

For tokenisation we use [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) from keras library.

In [0]:
txt = texts_sarcastic+ texts_serious

In [0]:
#your code here

tokenizer = Tokenizer(MAX_NB_WORDS)
tokenizer.fit_on_texts(txt)
word_to_id = tokenizer.word_index 
word_to_id = {k:v for k,v in  word_to_id.items()}
id_to_word = {v:k for k,v in word_to_id.items()}

Tokenize sarcastic texts and create tensor composed of tokens indexes. If a sentence shorter than MAX_SEQUENCE_LENGTH we pad it. If a sentence longer than MAX_SEQUENCE_LENGTH we cut it.

In [0]:
#your code here

sequences = tokenizer.texts_to_sequences(texts_sarcastic)
sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')

NB_WORDS = (min(tokenizer.num_words, len(word_to_id)) + 1 )

In [18]:
sequences

array([[16003,   354,  3166, ...,     0,     0,     0],
       [11426,  1593,  6417, ...,     0,     0,     0],
       [  471,   494,   321, ...,     0,     0,     0],
       ...,
       [ 5846,  7214,  5576, ...,   268,     0,     0],
       [ 1854,  1305,  3269, ...,     0,     0,     0],
       [  216,  3238,    20, ...,     0,     0,     0]], dtype=int32)

In [19]:
train_data

,is_sarcastic,headline,article_link
28580,False,pit bull lovers gather in washington,https://www.huffingtonpost.com/entry/pit-bull-...
14354,True,new tech-support caste arises in india,https://www.theonion.com/new-tech-support-cast...
19070,True,amazing original thing to become hated cliché ...,https://www.theonion.com/amazing-original-thin...
5045,False,united airlines temporarily suspends cargo tra...,https://www.huffingtonpost.com/entry/united-su...
24640,False,cuba and the united states: the long view,https://www.huffingtonpost.com/entry/cuba-and-...
...,...,...,...
21575,True,turnout lower than expected for gala central a...,https://www.theonion.com/turnout-lower-than-ex...
5390,True,retreating clinton campaign torches iowa town ...,https://politics.theonion.com/retreating-clint...
860,True,national weather service to give hurricanes fu...,https://www.theonion.com/national-weather-serv...
15795,True,christ returns for some of his old things,https://www.theonion.com/christ-returns-for-so...


Define batch generator to train a neural network

For padding sentences to max length we use [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)

In [124]:
def sent_generator(TRAIN_DATA_FILE, batchsize):
    # Create iterator that reads dataset file batch by batch 
    #your code here
    #Suffle the data file
    full = pd.read_csv(TRAIN_DATA_FILE)
    indexes = np.random.choice (range(len(full)),len(full),False)
    
    shuffled = full.iloc[indexes]

    reader = [shuffled.iloc[batchsize*i: batchsize*i +batch_size,:] for i in range(int(np.ceil(len(full)/batchsize)))]

    reader = pd.read_csv(TRAIN_DATA_FILE, chunksize= batchsize, iterator= True)
    
    for df in reader:

        # Read a column that contains headlines
        #your code here
        texts = list(df['headline'])
        if len (texts)<batchsize:
          texts = texts + [' ']*(batchsize - len(texts))

        # Tokenize texts and create padded tensor composed of tokens indexes
        sequences = tokenizer.texts_to_sequences(texts)
        sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
        
        # Return input-target pairs
        #your code here

        yield (sequences, sequences) 

print([a for a in sent_generator('train_data.csv',  4)][:1])

[(array([[ 3377,  3378,  5544,  2289,     4,   558,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [   11,  1237,   333,  9088, 19705,     4,  1908,     0,     0,
            0,     0,     0,     0,     0,     0],
       [ 1006,  2198,   229,     1,   703,  6761, 10628,     4,   210,
          576,     0,     0,     0,     0,     0],
       [  694,  1378,  5498,  5385,  6568,   644,     5,  4701,     0,
            0,     0,     0,     0,     0,     0]], dtype=int32), array([[ 3377,  3378,  5544,  2289,     4,   558,     0,     0,     0,
            0,     0,     0,     0,     0,     0],
       [   11,  1237,   333,  9088, 19705,     4,  1908,     0,     0,
            0,     0,     0,     0,     0,     0],
       [ 1006,  2198,   229,     1,   703,  6761, 10628,     4,   210,
          576,     0,     0,     0,     0,     0],
       [  694,  1378,  5498,  5385,  6568,   644,     5,  4701,     0,
            0,     0,     0,     0,     0,     0]], dtype=

Load pretrained GloVe vectors described in [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf) paper

In [64]:
embeddings_index = {}
with open('glove.6B.50d.txt', encoding='utf-8') as f:
    # read rows from file line by line
    for line in f:
        values = line.split()
        word = values[0] # Get word
        coefs = np.asarray(values[1:], dtype='float32') # Get elements of word's vector
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Create matrix from embedding vectors. Any row of the matrix is a word's vector. We get words from the dictionary word_to_id defined earlier.

In [126]:
glove_embedding_matrix = np.zeros((NB_WORDS, EMBEDDING_DIM)) # Create empty matrix (max number of tokens, dimension of the embedding vectors)
not_in_glove=0
for word, i in word_to_id.items():
    #your code here
    if i<NB_WORDS:
      try:
        glove_embedding_matrix[i] = embeddings_index[word]
      except:
        glove_embedding_matrix[i] = embeddings_index['unk']
        not_in_glove+=1
        continue
# compute number of words which there aren't in the GloVe vectors
print(f'Null word embeddings: {not_in_glove}')

Null word embeddings: 2465


Define parameters of the net

In [0]:
batch_size = 64
max_len = MAX_SEQUENCE_LENGTH
emb_dim = EMBEDDING_DIM
latent_dim = 32 # dimensionality of the hidden state in encoder and decoder RNN's
intermediate_dim = 96 # dimensionality of variational space into which we map encoder's hidden state
epsilon_std = 1.0 # standard deviation of gaussian noise
act = ELU() # activation function of projection layer

Encoder of the variational autoencoder. It based on bidirectional LSTM

We use following layers: [Input](https://www.tensorflow.org/api_docs/python/tf/keras/Input), [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding), [Bidirectional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional), [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM), [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout), [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), [ELU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ELU)

In [0]:
x = Input(batch_shape=(None, max_len)) # Input layer fo the net. 
# Write an embedding layer for the input sequences of indexes. 
# Use pretrained word embeddings as a embedding layer weights and don't update these weights

#your code here
x_embed = Embedding(NB_WORDS, emb_dim, weights=[glove_embedding_matrix],
                            input_length=max_len, trainable=False)(x)


# Bidirectional LSTM encoder

#your code here
h = Bidirectional(LSTM(intermediate_dim, return_sequences=False, recurrent_dropout=0.2), merge_mode='concat')(x_embed)



h = Dropout(0.2)(h) # Dropout for the BiLSTM layer to avoid overfitting 

# Fully-connected layer to map encoder hidden state into variational space

#your code here
h = act(Dense(intermediate_dim, activation='linear')(h))

h = Dropout(0.2)(h) # Dropout for the fully-connected layer to avoid overfitting 
z_mean = Dense(latent_dim)(h) # Fully-connected layer to map variational space into means space 
z_log_var = Dense(latent_dim)(h) # Fully-connected layer to map variational space into standard deviations space 

The mechanism for sampling hidden vectors from variational space

To apply it to our model we use [Lambda](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Lambda) layer

In [0]:
def sampling(args):
    # Vectors from means space and standard deviations space respectively
    z_mean, z_log_var = args
    # Sample random vectors from normal distribution with mean=0 and std=epsilon_std
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    
    #your code here
    
    # Get new hidden state for decoder using vectors from means, standard deviations and normal random spaces
    return z_mean + K.exp(z_log_var / 2) * epsilon

# Get hidden states for the decoder
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

Define decoder of the autoencoder

For this we use following layers: [RepeatVector](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RepeatVector), [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM), [TimeDistributed](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed), [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)

In [0]:
# Repeat the hidden state vector to form input sequence for decoder
repeated_context = RepeatVector(max_len)
# Decoder LSTM
decoder_h = LSTM(intermediate_dim, return_sequences=True, recurrent_dropout=0.2)
# Layer for mapping from hidden satates space to the space of dimension equal to size of vocabulary
decoder_mean = TimeDistributed(Dense(NB_WORDS, activation='linear'))
# Generated sequence
h_decoded = decoder_h(repeated_context(z))
# Decode every time step vector of the decoded sequence into space of dimension equal to size of vocabulary
x_decoded_mean = decoder_mean(h_decoded)

Define layer for loss computing

In [0]:
def zero_loss(y_true, y_pred):
    # Return tensor filled with ones with shape equal generated sequence shape
    return K.zeros_like(y_pred)

In [0]:
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)
        # Create tensor (batch_size, max_sequence_len) filled with ones to consider all elements of generated sequence 
        self.target_weights = tf.constant(np.ones((batch_size, max_len)), tf.float32)

    def vae_loss(self, x, x_decoded_mean):
        # Get tensor with similar shape as x
        labels = tf.cast(x, tf.int32)
        # Compute sequence reconstruction loss
        xent_loss = K.sum(tfa.seq2seq.sequence_loss(x_decoded_mean, labels,
                                                    weights=self.target_weights,
                                                    average_across_timesteps=False,
                                                    average_across_batch=False), axis=-1)
        # Compute KL-divergence as Variational loss 
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        # Composite loss (reconstruction loss + Variational loss)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0] # input sequence
        x_decoded_mean = inputs[1] # reconstructed sequence
        print(x.shape, x_decoded_mean.shape)
        loss = self.vae_loss(x, x_decoded_mean) # Compute loss of the model
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape
        return K.ones_like(x)

Assemble the model

To define model we use [Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model) from tensorflow.

To train model employ [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) optimization algorithm

In [133]:
# Create custom layer for loss computing
loss_layer = CustomVariationalLayer()([x, x_decoded_mean])

vae = Model(x, [loss_layer])
# Use Adam optimizer with learning rate = 0.01
opt = Adam(lr=0.005)
vae.compile(optimizer=opt, loss=[zero_loss])
# Show model structure
vae.summary()

(None, 15) (64, 15, 20001)
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 15, 50)       1000050     input_4[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 192)          112896      embedding_3[0][0]                
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 192)          0           bidirectional_3[0][0]            
_________________________________________________________________

Checkpoint function to save states of our model during training

In [0]:
def create_model_checkpoint(model_name):
    filepath = f"models/{model_name}.h5"
    directory = os.path.dirname(filepath)
    try:
        # Check if directory exists
        os.stat(directory)
    except:
        # If directory doesn't exist, create the directory
        os.mkdir(directory)
    # Save model states
    checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=False)
    return checkpointer

# Create model checkpointer
checkpointer = create_model_checkpoint('vae_seq2seq')

Train model, test model after each apoch and save model's state

In [0]:
sents = val_data['headline']
val_1_data = tokenizer.texts_to_sequences(sents)
val_1_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding = 'post')
val_1_data = val_1_data[:np.max([i for i in range(val_1_data.shape[0]) if i % 64 == 0]),:]

In [136]:
val_1_data

array([[16003,   354,  3166, ...,     0,     0,     0],
       [11426,  1593,  6417, ...,     0,     0,     0],
       [  471,   494,   321, ...,     0,     0,     0],
       ...,
       [ 6360,  3114,   230, ...,     0,     0,     0],
       [ 4070,     1,   961, ...,     0,     0,     0],
       [ 5846,  7214,  5576, ...,   268,     0,     0]], dtype=int32)

In [0]:
nb_epoch = 50 # number of epochs for model training
n_steps = train_data.shape[0] // batch_size # Number of steps per epoch
for counter in range(nb_epoch):
    print('-------epoch: ', counter, '-------')x
    # Train model. Test and save model after every epoch
    vae.fit_generator(sent_generator('train_data.csv', batch_size),
                      steps_per_epoch=n_steps, epochs=1, callbacks=[checkpointer],
                      validation_data=(val_1_data,val_1_data))
vae.save(r'vae_lstmFull32dim96hid.h5')

-------epoch:  0 -------
Epoch 1/1
335/335 [==============================] - 61s 181ms/step - loss: 87.0867 - val_loss: 83.2054

Epoch 00001: saving model to models/vae_seq2seq.h5
-------epoch:  1 -------
Epoch 1/1
335/335 [==============================] - 59s 177ms/step - loss: 77.2917 - val_loss: 82.4800

Epoch 00001: saving model to models/vae_seq2seq.h5
-------epoch:  2 -------
Epoch 1/1
189/335 [===============>..............] - ETA: 24s - loss: 75.0610

Assemble encoder and decoder for sentence generation sampled from variational space

In [0]:

# vae.load_weights("/content/models/vae_seq2seq.h5")

In [0]:
# Make separate encoder to encode input sentence
encoder = Model(x, z_mean)
# Input layer for decoder to decode vectors sampled from variational space 
decoder_input = Input(shape=(latent_dim,))
# Apply LSTM to decode hidden vector into sequence
_h_decoded = decoder_h(repeated_context(decoder_input))
# Decode every time step vector of the decoded sequence into space of dimension equal to size of vocabulary
_x_decoded_mean = decoder_mean(_h_decoded)
# Apply softmax to get most probable token
_x_decoded_mean = Activation('softmax')(_x_decoded_mean)
# Make decoderfor sempled sentences
generator = Model(decoder_input, _x_decoded_mean)

Generate sentence

In [0]:
# Dictionary maps indexes to words
index2word = {v: k for k, v in word_to_id.items()}
# Fit sentences from validation set into encoder
sent_encoded = encoder.predict(val_1_data, batch_size=16)
# Decode encoded sentences
x_test_reconstructed = generator.predict(sent_encoded)

sent_idx = 400
# Get words indexes with highest probability for the 400th sentence from validation set
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[sent_idx])
# Map indexes of generated sentence to words
word_list = list(np.vectorize(index2word.get)(reconstructed_indexes))
word_list = ' '.join([w for w in word_list if w])
print(word_list)
# Map indexes of input sentence to words
original_sent = list(np.vectorize(index2word.get)(val_1_data[sent_idx]))
original_sent = ' '.join([w for w in original_sent if w])
print(original_sent)

In [77]:
sent_encoded = encoder.predict(val_1_data[:1])
x_test_reconstructed = generator.predict(sent_encoded)
reconstructed_indexes = np.apply_along_axis(np.argmax, 1, x_test_reconstructed[0])
original_sent = list(np.vectorize(id_to_word.get)(val_1_data[0]))
original_sent = ' '.join([w for w in original_sent if w])
print(original_sent)

thirtysomething scientists unveil doomsday clock of hair loss None None None None None None None


In [95]:
original_sent

'thirtysomething scientists unveil doomsday clock of hair loss'

In [78]:
s = list(np.vectorize(id_to_word.get)(reconstructed_indexes))
s

['guitar',
 'guitar',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri',
 'terri']

In [79]:
' '.join([w for w in s if w])

'guitar guitar terri terri terri terri terri terri terri terri terri terri terri terri terri'